In [126]:
import melt_functions as ice_melt
import numpy as np
import xarray as xr
import scipy.io as sio
from plot_icebergshape import plot_icebergshape
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d
from matplotlib import cm,colors
import pickle
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
import pandas as pd
import dataframe_image as dfi


In [211]:


dt = 50

psw = 1024 #kg m3
csw = 3974 #J kg-1 C-1
day2sec = 86400
# Volume_test = 28e3 * 5e3 * 300 #km3 Helheim Fjord test
Volume_test = 148461041 * 300 #km3 Helheim Fjord test

coeff_1_path = '/media/laserglaciers/upernavik/iceberg_py/outfiles/helheim/Qib/coeff_1_melt_rates/'
coeff_4_path = '/media/laserglaciers/upernavik/iceberg_py/outfiles/helheim/Qib/coeff_4_melt_rates/'

coeff_1_list = sorted([nc for nc in os.listdir(coeff_1_path) if nc.endswith('nc')])
coeff_4_list = sorted([nc for nc in os.listdir(coeff_4_path) if nc.endswith('nc')])


constant_tf_57 = 5.73 # from Slater 2022 nature geoscience
constant_tf_67 = 6.67 # from Slater 2022 nature geoscience
constant_tf_76 = 7.62 # from Slater 2022 nature geoscience

dQ_dt_HEL_57 = psw * csw * ( (Volume_test * constant_tf_57) / (dt * day2sec) )
dQ_dt_HEL_67 = psw * csw * ( (Volume_test * constant_tf_67) / (dt * day2sec) )
dQ_dt_HEL_76 = psw * csw * ( (Volume_test * constant_tf_76) / (dt * day2sec) )

os.chdir(coeff_1_path)
dQ_dt_list = [dQ_dt_HEL_57, dQ_dt_HEL_67, dQ_dt_HEL_76]
dQ_dt_dict = {'5.73':dQ_dt_HEL_57,
              '6.67':dQ_dt_HEL_67,
              '7.62':dQ_dt_HEL_76
    
}

tfs = [dQ_dt_HEL_57, constant_tf_67, dQ_dt_HEL_76]

TF_dict = {'5.73':constant_tf_57,
              '6.67':constant_tf_67,
              '7.62':constant_tf_76
    
}

urels = [0.01, 0.05, 0.10] #m s-1

urels_dict = {'01': 0.01,
              '05' : 0.05,
              '1' : 0.10
}

cols = ['coeff', 'urel', 'tf', 'dt', 'Qib', 'Qaww', 'melt_rate_avg_m3s', 'melt_rate_mday', 'percentage']



In [212]:
# pd.options.display.latex.repr = True
# l = df_concat.to_latex(
#     index=False,  # To not include the DataFrame index as a column in the table
#     position="htbp",  # The preferred positions where the table should be placed in the document ('here', 'top', 'bottom', 'page')
#     column_format="|l|l|l|l|l|l|l|",  # The format of the columns: left-aligned with vertical lines between them
#     escape=True,  # Disable escaping LaTeX special characters in the DataFrame
#     float_format="{:0.2f}".format  # Formats floats to two decimal places
# )
# l2 = df_concat.to_latex(
#     index = False,
#     float_format="{:0.2f}".format  # Formats floats to two decimal places
# )

In [213]:
# print(l2)

In [214]:
# dfi.export(df_concat, '/media/laserglaciers/upernavik/iceberg_py/figs/df_concat_vol_28km_length_melt_rates.png')

In [215]:
# pd.options.display.latex.repr = True

In [216]:
# help(dfi.export)

In [217]:
# df_concat.to_csv('/media/laserglaciers/upernavik/iceberg_py/Qib_test.csv')

In [270]:
coeff_1_path = '/media/laserglaciers/upernavik/iceberg_py/outfiles/helheim/Qib_melt_flux_v2/coeff_1/'
coeff_4_path = '/media/laserglaciers/upernavik/iceberg_py/outfiles/helheim/Qib_melt_flux_v2/coeff_4/'

coeff_1_list = sorted([nc for nc in os.listdir(coeff_1_path) if nc.endswith('nc')])
coeff_4_list = sorted([nc for nc in os.listdir(coeff_4_path) if nc.endswith('nc')])


constant_tf_57 = 5.73 # from Slater 2022 nature geoscience
constant_tf_67 = 6.67 # from Slater 2022 nature geoscience
constant_tf_76 = 7.62 # from Slater 2022 nature geoscience

dQ_dt_HEL_57 = psw * csw * ( (Volume_test * constant_tf_57) / (dt * day2sec) )
dQ_dt_HEL_67 = psw * csw * ( (Volume_test * constant_tf_67) / (dt * day2sec) )
dQ_dt_HEL_76 = psw * csw * ( (Volume_test * constant_tf_76) / (dt * day2sec) )

os.chdir(coeff_1_path)
dQ_dt_list = [dQ_dt_HEL_57, dQ_dt_HEL_67, constant_tf_76]

tfs = [constant_tf_57, constant_tf_67, constant_tf_76]
urels = [0.01, 0.05, 0.10] #m s-1

# cols = ['date', 'coeff', 'urel', 'tf', 'dt', 'Qib', 'Qaww', 'melt_rate_avg_m3s', 'melt_rate_mday', 'percentage']
cols = ['date', 'coeff', 'urel', 'tf', 'dt', 'Qib', 'Qaww', 'melt_flux_m3s', 'percentage']
coef_1_dict = {}


series_list1 = []
for i,nc in enumerate(coeff_1_list):
    
    Qib = xr.open_dataset(nc)
    Qib_val = Qib.Qib.data
    date = nc[:10]
    date = pd.to_datetime(date)
    TF = nc.split('_')[6]
    urel_val = nc.split('_')[-1].split('.')[1]
    
    percentage = Qib_val/dQ_dt_dict[TF]
    print(f'coeff 1: {percentage:.2f}')
    print(f'{date}')
    coef_1_dict['date'] = date
    coef_1_dict['coeff'] = 1
    coef_1_dict['urel'] = urels_dict[urel_val]
    coef_1_dict['tf'] = TF_dict[TF]
    coef_1_dict['dt'] = 50
    coef_1_dict['Qib'] = (Qib_val)
    coef_1_dict['Qaww'] = (dQ_dt_dict[TF])
    # coef_1_dict['melt_rate_avg_m3s'] = Qib.melt_rate_avg.data
    coef_1_dict['melt_flux_m3s'] = Qib.melt_rate_intergrated.data
    
    # coef_1_dict['percentage'] = percentage*100
    
    series = pd.Series(coef_1_dict)
    series_list1.append(series)

df_1 = pd.DataFrame(series_list1, columns=cols)   
os.chdir(coeff_4_path)

coef_4_dict = {}
series_list4 = []
for i,nc in enumerate(coeff_4_list):
    
    Qib = xr.open_dataset(nc)
    Qib_val = Qib.Qib.data
    date = nc[:10]
    date = pd.to_datetime(date)
    TF_4 = nc.split('_')[6]
    urel_val = nc.split('_')[-1].split('.')[1]
    
    percentage = Qib_val/dQ_dt_dict[TF]
    print(f'coeff 4: {percentage:.2f}')
    print(f'{TF}')
    coef_4_dict['date'] = date
    coef_4_dict['coeff'] = 4
    coef_4_dict['urel'] = urels_dict[urel_val]
    coef_4_dict['tf'] = TF_dict[TF_4]
    coef_4_dict['dt'] = 50
    coef_4_dict['Qib'] = (Qib_val)
    coef_4_dict['Qaww'] = (dQ_dt_dict[TF_4])
    # coef_4_dict['melt_rate_avg_m3s'] = Qib.melt_rate_avg.data
    coef_4_dict['melt_flux_m3s'] = Qib.melt_rate_intergrated.data

    
    # coef_4_dict['percentage'] = percentage*100
    
    series4 = pd.Series(coef_4_dict)
    series_list4.append(series4)
    


df_1 = pd.DataFrame(series_list1, columns=cols)
df_4 = pd.DataFrame(series_list4, columns=cols)

df_concat = pd.concat([df_1, df_4],ignore_index=True)


coeff 1: 0.02
2016-04-24 00:00:00
coeff 1: 0.10
2016-04-24 00:00:00
coeff 1: 0.20
2016-04-24 00:00:00
coeff 1: 0.02
2018-05-04 00:00:00
coeff 1: 0.10
2018-05-04 00:00:00
coeff 1: 0.20
2018-05-04 00:00:00
coeff 1: 0.01
2020-09-03 00:00:00
coeff 1: 0.06
2020-09-03 00:00:00
coeff 1: 0.13
2020-09-03 00:00:00
coeff 1: 0.02
2023-07-27 00:00:00
coeff 1: 0.11
2023-07-27 00:00:00
coeff 1: 0.22
2023-07-27 00:00:00
coeff 1: 0.02
2024-08-22 00:00:00
coeff 1: 0.11
2024-08-22 00:00:00
coeff 1: 0.23
2024-08-22 00:00:00
coeff 4: 0.06
7.62
coeff 4: 0.34
7.62
coeff 4: 0.79
7.62
coeff 4: 0.06
7.62
coeff 4: 0.34
7.62
coeff 4: 0.80
7.62
coeff 4: 0.04
7.62
coeff 4: 0.22
7.62
coeff 4: 0.51
7.62
coeff 4: 0.06
7.62
coeff 4: 0.38
7.62
coeff 4: 0.87
7.62
coeff 4: 0.07
7.62
coeff 4: 0.39
7.62
coeff 4: 0.92
7.62


In [271]:
df_concat

,date,coeff,urel,tf,dt,Qib,Qaww,melt_flux_m3s,percentage
0,2016-04-24,1,0.01,5.73,50,4590469496.605303,2.403989e+11,59.0501395448605,NaN
1,2016-04-24,1,0.05,6.67,50,27360364477.546455,2.798361e+11,351.7303839870241,NaN
2,2016-04-24,1,0.10,7.62,50,63778691572.90796,3.196928e+11,817.114312996017,NaN
3,2018-05-04,1,0.01,5.73,50,4618572949.833515,2.403989e+11,49.595267478804075,NaN
4,2018-05-04,1,0.05,6.67,50,27527992303.063637,2.798361e+11,295.4247575581755,NaN
5,2018-05-04,1,0.10,7.62,50,64169693315.96497,3.196928e+11,686.5788687324133,NaN
6,2020-09-03,1,0.01,5.73,50,2984399096.9485736,2.403989e+11,45.308572586859434,NaN
7,2020-09-03,1,0.05,6.67,50,17786426809.45553,2.798361e+11,269.86749611497316,NaN
8,2020-09-03,1,0.10,7.62,50,41458521025.296844,3.196928e+11,626.9082322153548,NaN
9,2023-07-27,1,0.01,5.73,50,5042833550.407843,2.403989e+11,86.08300923926814,NaN


In [272]:
c4 = df_concat['coeff']==4
df_concat.loc[c4]['melt_flux_m3s'].mean()

1590.659791903073

In [273]:
c1 = df_concat['coeff']==1
df_concat.loc[c1]['melt_flux_m3s'].mean()

399.87605456052773

In [274]:
df_concat.loc[c4]['Qib'].mean()

124565737448.89694

In [275]:
df_concat.loc[c1]['Qib'].mean()

31321189443.486034

In [276]:
c4_urel05 = (df_concat['coeff']==4) & (df_concat['urel'] == 0.05)
df_concat.loc[c4_urel05]['Qib'].mean()

107072299500.05984

In [277]:
c1_urel05 = (df_concat['coeff']==1) & (df_concat['urel'] == 0.05)
df_concat.loc[c1_urel05]['Qib'].mean()

26855352464.081364

In [278]:
df_concat.loc[c1_urel05]

,date,coeff,urel,tf,dt,Qib,Qaww,melt_flux_m3s,percentage
1,2016-04-24,1,0.05,6.67,50,27360364477.546455,2.798361e+11,351.7303839870241,NaN
4,2018-05-04,1,0.05,6.67,50,27527992303.063637,2.798361e+11,295.4247575581755,NaN
7,2020-09-03,1,0.05,6.67,50,17786426809.45553,2.798361e+11,269.86749611497316,NaN
10,2023-07-27,1,0.05,6.67,50,30057899160.03247,2.798361e+11,512.7209894074392,NaN
13,2024-08-22,1,0.05,6.67,50,31544079570.308735,2.798361e+11,287.9335799052789,NaN


In [279]:
df_concat.loc[c1_urel05]['melt_flux_m3s'].mean()

343.5354413945782

In [280]:
c1_urel01 = (df_concat['coeff']==1) & (df_concat['urel'] == 0.01) & (df_concat['tf'] == 5.73)
df_concat.loc[c1_urel01]

,date,coeff,urel,tf,dt,Qib,Qaww,melt_flux_m3s,percentage
0,2016-04-24,1,0.01,5.73,50,4590469496.605303,2.403989e+11,59.0501395448605,NaN
3,2018-05-04,1,0.01,5.73,50,4618572949.833515,2.403989e+11,49.595267478804075,NaN
6,2020-09-03,1,0.01,5.73,50,2984399096.9485736,2.403989e+11,45.308572586859434,NaN
9,2023-07-27,1,0.01,5.73,50,5042833550.407843,2.403989e+11,86.08300923926814,NaN
12,2024-08-22,1,0.01,5.73,50,5291971539.396828,2.403989e+11,48.33442432671509,NaN


In [281]:
df_concat.loc[c1_urel01]['melt_flux_m3s'].mean()

57.67428263530145

In [282]:
c1_urel10 = (df_concat['coeff']==1) & (df_concat['urel'] == 0.10) & (df_concat['tf'] == 7.62)
df_concat.loc[c1_urel10]

,date,coeff,urel,tf,dt,Qib,Qaww,melt_flux_m3s,percentage
2,2016-04-24,1,0.1,7.62,50,63778691572.90796,3.196928e+11,817.114312996017,NaN
5,2018-05-04,1,0.1,7.62,50,64169693315.96497,3.196928e+11,686.5788687324133,NaN
8,2020-09-03,1,0.1,7.62,50,41458521025.296844,3.196928e+11,626.9082322153548,NaN
11,2023-07-27,1,0.1,7.62,50,70069504093.80724,3.196928e+11,1191.8263986286022,NaN
14,2024-08-22,1,0.1,7.62,50,73536422690.71463,3.196928e+11,669.664385686132,NaN


In [283]:
df_concat.loc[c1_urel10]['melt_flux_m3s'].mean()

798.4184396517039

In [284]:
c4_urel01 = (df_concat['coeff']==4) & (df_concat['urel'] == 0.01) & (df_concat['tf'] == 5.73)
df_concat.loc[c4_urel01]

,date,coeff,urel,tf,dt,Qib,Qaww,melt_flux_m3s,percentage
15,2016-04-24,4,0.01,5.73,50,18361877986.42121,2.403989e+11,236.200558179442,NaN
18,2018-05-04,4,0.01,5.73,50,18474291799.33406,2.403989e+11,198.3810699152163,NaN
21,2020-09-03,4,0.01,5.73,50,11937596387.794294,2.403989e+11,181.23429034743774,NaN
24,2023-07-27,4,0.01,5.73,50,20171334201.63137,2.403989e+11,344.33203695707255,NaN
27,2024-08-22,4,0.01,5.73,50,21167886157.58731,2.403989e+11,193.33769730686035,NaN


In [285]:
df_concat.loc[c4_urel01]['melt_flux_m3s'].mean()

230.6971305412058

In [286]:
c4_urel05 = (df_concat['coeff']==4) & (df_concat['urel'] == 0.05) & (df_concat['tf'] == 6.67)
df_concat.loc[c4_urel05]

,date,coeff,urel,tf,dt,Qib,Qaww,melt_flux_m3s,percentage
16,2016-04-24,4,0.05,6.67,50,109042155805.81735,2.798361e+11,1401.7093679534992,NaN
19,2018-05-04,4,0.05,6.67,50,109600097974.74391,2.798361e+11,1177.2140702222136,NaN
22,2020-09-03,4,0.05,6.67,50,70823200035.88377,2.798361e+11,1075.4146611794993,NaN
25,2023-07-27,4,0.05,6.67,50,120072897637.86368,2.798361e+11,2045.877202728424,NaN
28,2024-08-22,4,0.05,6.67,50,125823146045.9905,2.798361e+11,1148.5827654505072,NaN


In [287]:
df_concat.loc[c4_urel05]['melt_flux_m3s'].mean()

1369.7596135068286

In [288]:
c4_urel10 = (df_concat['coeff']==4) & (df_concat['urel'] == 0.1) & (df_concat['tf'] == 7.62)
df_concat.loc[c4_urel10]

,date,coeff,urel,tf,dt,Qib,Qaww,melt_flux_m3s,percentage
17,2016-04-24,4,0.1,7.62,50,253195647504.3717,3.196928e+11,3252.5776765814053,NaN
20,2018-05-04,4,0.1,7.62,50,254899692929.49716,3.196928e+11,2733.986660465604,NaN
23,2020-09-03,4,0.1,7.62,50,163984249757.5382,3.196928e+11,2493.8631717481653,NaN
26,2023-07-27,4,0.1,7.62,50,278378298638.9765,3.196928e+11,4710.929830813608,NaN
29,2024-08-22,4,0.1,7.62,50,292553688870.0029,3.196928e+11,2666.255818697142,NaN


In [289]:
df_concat.loc[c4_urel10]['melt_flux_m3s'].mean()

3171.522631661185

In [290]:
df_1

,date,coeff,urel,tf,dt,Qib,Qaww,melt_flux_m3s,percentage
0,2016-04-24,1,0.01,5.73,50,4590469496.605303,2.403989e+11,59.0501395448605,NaN
1,2016-04-24,1,0.05,6.67,50,27360364477.546455,2.798361e+11,351.7303839870241,NaN
2,2016-04-24,1,0.10,7.62,50,63778691572.90796,3.196928e+11,817.114312996017,NaN
3,2018-05-04,1,0.01,5.73,50,4618572949.833515,2.403989e+11,49.595267478804075,NaN
4,2018-05-04,1,0.05,6.67,50,27527992303.063637,2.798361e+11,295.4247575581755,NaN
5,2018-05-04,1,0.10,7.62,50,64169693315.96497,3.196928e+11,686.5788687324133,NaN
6,2020-09-03,1,0.01,5.73,50,2984399096.9485736,2.403989e+11,45.308572586859434,NaN
7,2020-09-03,1,0.05,6.67,50,17786426809.45553,2.798361e+11,269.86749611497316,NaN
8,2020-09-03,1,0.10,7.62,50,41458521025.296844,3.196928e+11,626.9082322153548,NaN
9,2023-07-27,1,0.01,5.73,50,5042833550.407843,2.403989e+11,86.08300923926814,NaN


In [291]:
df_4

,date,coeff,urel,tf,dt,Qib,Qaww,melt_flux_m3s,percentage
0,2016-04-24,4,0.01,5.73,50,18361877986.42121,2.403989e+11,236.200558179442,NaN
1,2016-04-24,4,0.05,6.67,50,109042155805.81735,2.798361e+11,1401.7093679534992,NaN
2,2016-04-24,4,0.10,7.62,50,253195647504.3717,3.196928e+11,3252.5776765814053,NaN
3,2018-05-04,4,0.01,5.73,50,18474291799.33406,2.403989e+11,198.3810699152163,NaN
4,2018-05-04,4,0.05,6.67,50,109600097974.74391,2.798361e+11,1177.2140702222136,NaN
5,2018-05-04,4,0.10,7.62,50,254899692929.49716,3.196928e+11,2733.986660465604,NaN
6,2020-09-03,4,0.01,5.73,50,11937596387.794294,2.403989e+11,181.23429034743774,NaN
7,2020-09-03,4,0.05,6.67,50,70823200035.88377,2.798361e+11,1075.4146611794993,NaN
8,2020-09-03,4,0.10,7.62,50,163984249757.5382,3.196928e+11,2493.8631717481653,NaN
9,2023-07-27,4,0.01,5.73,50,20171334201.63137,2.403989e+11,344.33203695707255,NaN


In [292]:
df_concat.loc[c4_urel05]['percentage'].mean()

nan

In [293]:
df_concat.loc[c4_urel05].Qib/df_concat.loc[c4_urel05].Qaww

16    0.389664
19    0.391658
22    0.253088
25    0.429083
28    0.449632
dtype: object

In [295]:
df_concat.loc[c4_urel05]

,date,coeff,urel,tf,dt,Qib,Qaww,melt_flux_m3s,percentage
16,2016-04-24,4,0.05,6.67,50,109042155805.81735,2.798361e+11,1401.7093679534992,NaN
19,2018-05-04,4,0.05,6.67,50,109600097974.74391,2.798361e+11,1177.2140702222136,NaN
22,2020-09-03,4,0.05,6.67,50,70823200035.88377,2.798361e+11,1075.4146611794993,NaN
25,2023-07-27,4,0.05,6.67,50,120072897637.86368,2.798361e+11,2045.877202728424,NaN
28,2024-08-22,4,0.05,6.67,50,125823146045.9905,2.798361e+11,1148.5827654505072,NaN


In [298]:
df_concat['percentage'] = (df_concat.Qib/df_concat.Qaww)*100

In [299]:
df_concat

,date,coeff,urel,tf,dt,Qib,Qaww,melt_flux_m3s,percentage
0,2016-04-24,1,0.01,5.73,50,4590469496.605303,2.403989e+11,59.0501395448605,1.909522
1,2016-04-24,1,0.05,6.67,50,27360364477.546455,2.798361e+11,351.7303839870241,9.777284
2,2016-04-24,1,0.10,7.62,50,63778691572.90796,3.196928e+11,817.114312996017,19.949996
3,2018-05-04,1,0.01,5.73,50,4618572949.833515,2.403989e+11,49.595267478804075,1.921212
4,2018-05-04,1,0.05,6.67,50,27527992303.063637,2.798361e+11,295.4247575581755,9.837186
5,2018-05-04,1,0.10,7.62,50,64169693315.96497,3.196928e+11,686.5788687324133,20.072301
6,2020-09-03,1,0.01,5.73,50,2984399096.9485736,2.403989e+11,45.308572586859434,1.241436
7,2020-09-03,1,0.05,6.67,50,17786426809.45553,2.798361e+11,269.86749611497316,6.356017
8,2020-09-03,1,0.10,7.62,50,41458521025.296844,3.196928e+11,626.9082322153548,12.968239
9,2023-07-27,1,0.01,5.73,50,5042833550.407843,2.403989e+11,86.08300923926814,2.097694


In [302]:
df_concat.loc[c4_urel05].percentage.mean()

38.26251096273097

In [ ]:
df_concat.to_csv('/media/laserglaciers/upernavik/iceberg_py/csv/'